# Модуль json

Официальная документация [здесь](https://docs.python.org/3/library/json.html)

JSON (JavaScript Object Notation), указанный в RFC 7159 (который заменяет RFC 4627) и ECMA-404, представляет собой легкий формат обмена данными, основанный на синтаксисе объектного литерала JavaScript (хотя он и не является строгим подмножеством JavaScript).

Подробнее о RFC [здесь](https://tools.ietf.org/html/rfc7159.html)

Модуль json предоставляет API, аналогичный pickle, для преобразования объектов Python в памяти в сериализованное представление, известное как JavaScript Object Notation (JSON). В отличие от pickle, JSON имеет преимущество в наличии реализаций на многих языках (особенно JavaScript). Он наиболее широко используется для связи между веб-сервером и клиентом в REST API, но также полезен для других потребностей коммуникации между приложениями.

In [1]:
import json

## Кодирование и декодирование простых типов данных

Кодировщик по умолчанию понимает собственные типы Python (str, int, float, list, tuple и dict).

Класс json.JSONDecoder(object_hook=None, parse_float=None, parse_int=None, parse_constant=None, strict=True, object_pairs_hook=None) - простой декодер JSON.

Выполняет следующие преобразования при декодировании:

|      JSON     | Python |
|:-------------:|:------:|
| object        | dict   |
| array         | list   |
| string        | str    |
| number (int)  | int    |
| number (real) | float  |
| true          | True   |
| false         | False  |
| null          | None   |

Он также понимает NaN, Infinity, и -Infinity как соответствующие значения float, которые находятся за пределами спецификации JSON.

Класс json.JSONEncoder(skipkeys=False, ensure_ascii=True, check_circular=True, allow_nan=True, sort_keys=False, indent=None, separators=None, default=None)

Расширяемый кодировщик JSON для структур данных Python. Поддерживает следующие объекты и типы данных по умолчанию:

|    Python   |  JSON  |
|:-----------:|:------:|
| dict        | object |
| list, tuple | array  |
| str         | string |
| int, float  | number |
| True        | true   |
| False       | false  |
| None        | null   |

In [2]:
import json

data = [{'a': 'A', 'b': (2, 4), 'c': 3.0}]
print('DATA:', repr(data))

data_string = json.dumps(data)
print('JSON:', data_string)

DATA: [{'a': 'A', 'b': (2, 4), 'c': 3.0}]
JSON: [{"a": "A", "b": [2, 4], "c": 3.0}]


Значения кодируются способом, внешне похожим на вывод питоновского repr().

Кодирование, а затем повторное декодирование может не дать точно такого же типа объекта.

In [3]:
import json

data = [{'a': 'A', 'b': (2, 4), 'c': 3.0}]
print('DATA   :', data)

data_string = json.dumps(data)
print('ENCODED:', data_string)

decoded = json.loads(data_string)
print('DECODED:', decoded)

print('ORIGINAL:', type(data[0]['b']))
print('DECODED :', type(decoded[0]['b']))

DATA   : [{'a': 'A', 'b': (2, 4), 'c': 3.0}]
ENCODED: [{"a": "A", "b": [2, 4], "c": 3.0}]
DECODED: [{'a': 'A', 'b': [2, 4], 'c': 3.0}]
ORIGINAL: <class 'tuple'>
DECODED : <class 'list'>


В частности, как мы видим, кортежи становятся списками.

## Удобочитаемость против компактности

Еще одно преимущество JSON перед pickle заключаются в результатах человеческого восприятия. Функция dumps() принимает несколько аргументов, чтобы сделать вывод еще приятнее. Например, флаг sort_keys указывает кодировщику выводить ключи словаря в отсортированном, а не случайном порядке.

In [4]:
import json

data = [{'a': 'A', 'b': (2, 4), 'c': 3.0}]
print('DATA:', repr(data))

unsorted = json.dumps(data)
print('JSON:', json.dumps(data))
print('SORT:', json.dumps(data, sort_keys=True))

first = json.dumps(data, sort_keys=True)
second = json.dumps(data, sort_keys=True)

print('UNSORTED MATCH:', unsorted == first)
print('SORTED MATCH  :', first == second)

DATA: [{'a': 'A', 'b': (2, 4), 'c': 3.0}]
JSON: [{"a": "A", "b": [2, 4], "c": 3.0}]
SORT: [{"a": "A", "b": [2, 4], "c": 3.0}]
UNSORTED MATCH: True
SORTED MATCH  : True


Сортировка облегчает сканирование результатов на глаз, а также позволяет сравнивать вывод JSON в тестах.

Для сильно вложенных структур данных укажите значение отступа, чтобы выходные данные также были хорошо отформатированы.

In [5]:
import json

data = [{'a': 'A', 'b': (2, 4), 'c': 3.0}]
print('DATA:', repr(data))

print('NORMAL:', json.dumps(data, sort_keys=True))
print('INDENT:', json.dumps(data, sort_keys=True, indent=2))

DATA: [{'a': 'A', 'b': (2, 4), 'c': 3.0}]
NORMAL: [{"a": "A", "b": [2, 4], "c": 3.0}]
INDENT: [
  {
    "a": "A",
    "b": [
      2,
      4
    ],
    "c": 3.0
  }
]


Когда indent является неотрицательным целым числом, выходные данные больше похожи на pprint, с начальными пробелами для каждого уровня структуры данных, соответствующего уровню отступа.

Однако такой подробный вывод увеличивает количество байтов, необходимых для передачи того же объема данных, поэтому он не предназначен для использования в производственной среде. На самом деле, можно настроить параметры разделения данных в кодированном выводе, чтобы сделать его еще более компактным, чем по умолчанию.

In [19]:
import json

data = [{'a': 'A', 'b': (2, 4), 'c': 3.0}]
print('DATA:', repr(data))

print('repr(data)             :', len(repr(data)))

plain_dump = json.dumps(data)
print('dumps(data)            :', len(plain_dump))

small_indent = json.dumps(data, indent=2)
print('dumps(data, indent=2)  :', len(small_indent))

with_separators = json.dumps(data, separators=(',', ':'))
print('dumps(data, separators):', len(with_separators))

DATA: [{'a': 'A', 'b': (2, 4), 'c': 3.0}]
repr(data)             : 35
dumps(data)            : 35
dumps(data, indent=2)  : 73
dumps(data, separators): 29


Аргумент separators для dumps() должен быть кортежем, содержащим строки для разделения элементов в списке и ключей от значений в словаре. По умолчанию это (', ', ': '). Удаляя пробелы, получается более компактный вывод.

## Кодирование словарей

Формат JSON предполагает, что ключи к словарю будут строками. Попытка кодировать словарь с нестроковыми типами в качестве ключей приводит к ошибке TypeError. Один из способов обойти это ограничение - сказать кодировщику пропустить нестроковые ключи с помощью аргумента skipkeys

In [7]:
import json

data = [{'a': 'A', 'b': (2, 4), 'c': 3.0, ('d',): 'D tuple'}]

print('First attempt')
try:
    print(json.dumps(data))
except TypeError as err:
    print('ERROR:', err)

print()
print('Second attempt')
print(json.dumps(data, skipkeys=True))

First attempt
ERROR: keys must be str, int, float, bool or None, not tuple

Second attempt
[{"a": "A", "b": [2, 4], "c": 3.0}]


Вместо того чтобы вызывать исключение, нестроковый ключ игнорируется.

[Полезный материал для понимания темы](https://python-scripts.com/json)

# Задание

Убедитесь, что кодирование и последующее декодирование не дает одного и того же результата.